In [259]:
import pandas as pd
import numpy as np
import re
import os
from typing import List, Dict
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydrive.files import GoogleDriveFile
from datetime import datetime

In [2]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=900063863517-ci8cqlc3givum789c4ulpgj015jbkfmr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


The first function is taken from https://stackoverflow.com/questions/34101427/accessing-folders-subfolders-and-subfiles-using-pydrive-python and slightly changed as we're dealing with a finite set.

In [249]:
def list_folder(parent: str) -> List:
    file_list = []
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % parent}).GetList()
    for f in file_list:
        if 'mimeType' in f.keys():
            if f['mimeType'] == 'application/vnd.google-apps.folder': # if folder
                file_list.append({"id": f['id'],"title": f['title'],"list": ListFolder(f['id'])})
            else:
                file_list.append({"title": f['title']})
    
    return file_list[2]


def get_files_from_gfolder(file_list: List, folder: str) -> List:
    data_folder = [data_folder for data_folder in sample_data_folders[0]['list']
                 if data_folder['title'] == folder]
    file_names = [file['title'] for file in data_folder[0]['list']]
    
    return file_names

In [250]:
folder_structure = list_folder('root')
relevant_data_folders = ['pageviews', 'events', 'commerce']

In [142]:
def get_gdrive_csv_file_into_df(filename: str) -> pd.DataFrame:
    query = "title = '" + filename + "'"
    file_meta = drive.ListFile({'q': query}).GetList()
    file_id = file_meta[0]['id']
    gdrive_file = drive.CreateFile({'id': file_id})
    gdrive_file.GetContentFile('intermediate_data.gz')
    csv_data = pd.read_csv('intermediate_data.gz', low_memory=False)
    os.remove('intermediate_data.gz')
    
    return csv_data

In [255]:
folder_structure = list_folder('root')
df_dict = {}
for data_folder in relevant_data_folders:
    print(data_folder)
    df_dict[data_folder] = pd.DataFrame()
    folder_file_list = get_files_from_gfolder(folder_structure['list'], data_folder)
    for index in range(0, len(folder_file_list)):
        if not index % int(round(len(folder_file_list) / 10, 0)) or index == len(folder_file_list) - 1:
            print(index / len(folder_file_list))
        file_df = get_gdrive_csv_file_into_df(folder_file_list[index])
        df_dict[data_folder] = df_dict[data_folder].append(file_df)  

pageviews
0.0
0.10344827586206896
0.20689655172413793
0.3103448275862069
0.41379310344827586
0.5172413793103449
0.6206896551724138
0.7241379310344828
0.8275862068965517
0.9310344827586207
0.9655172413793104
events
0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
0.9666666666666667
commerce
0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
0.9666666666666667


In [257]:
[{key: len(value)} for key, value in df_dict.items()]

[{'pageviews': 9236063}, {'events': 2609406}, {'commerce': 41779}]

In [281]:
df_dict['pageviews'].head()

,_article,action,article_id,category,host,name,remp_session_id,signed_in,social,tags,time,title,token,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
0,0,load,NaN,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,NaN,NaN,NaN,NaN,1515801600076000000,NaN,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/autor/bella/,NaN,2dfb3791-a51c-4c7a-819d-f7a94a29c015,NaN,NaN,NaN,NaN
1,1,load,294893.0,slovensko,npress-influxdb-n2.ewhs.bntb.net,pageviews,NaN,NaN,NaN,vybuchy-v-parizi,1515801600078000000,NaN,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/294893/fico-bezpecnost-u-na...,NaN,2dfb3791-a51c-4c7a-819d-f7a94a29c015,NaN,NaN,NaN,NaN
2,1,load,544744.0,slovensko,npress-influxdb-n2.ewhs.bntb.net,pageviews,NaN,NaN,NaN,NaN,1515801600078000000,NaN,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/544744/ciznar-a-matovic-si-...,NaN,2dfb3791-a51c-4c7a-819d-f7a94a29c015,NaN,NaN,NaN,NaN
3,1,load,704078.0,komentare,npress-influxdb-n2.ewhs.bntb.net,pageviews,NaN,NaN,NaN,NaN,1515801600078000000,NaN,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/704078/za-boha-a-za-narod-b...,NaN,2dfb3791-a51c-4c7a-819d-f7a94a29c015,NaN,NaN,NaN,NaN
4,1,load,953687.0,nezaradene,npress-influxdb-n2.ewhs.bntb.net,pageviews,NaN,NaN,NaN,rozhovory,1515801600078000000,NaN,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/953687/ako-evanjelici-sami-...,NaN,2dfb3791-a51c-4c7a-819d-f7a94a29c015,NaN,NaN,NaN,NaN


Reshaping data types

In [374]:
# df_dict['pageviews'].rename(columns={'_article': 'is_article'}, inplace=True)
df_dict['pageviews']['is_article'] = df_dict['pageviews']['is_article'].astype(bool)
df_dict['pageviews']['article_id'] = df_dict['pageviews']['article_id'].astype(str)
df_dict['pageviews']['time'] = df_dict['pageviews']['time'].astype(datetime)
df_dict['pageviews']['article_id'] = df_dict['pageviews']['article_id'].str.replace('nan', '')
df_dict['pageviews']['remp_session_id'].fillna('', inplace = True)
# df_dict['pageviews']['signed_in'].fillna('', inplace = True)
df_dict['pageviews']['social'].fillna('', inplace = True)
df_dict['pageviews']['tags'].fillna('', inplace = True)
df_dict['pageviews']['title'].fillna('', inplace = True)
df_dict['pageviews']['user_agent'].fillna('', inplace = True)
[df_dict['pageviews'][column].fillna('', inplace = True) for column in df_dict['pageviews'].columns if re.search('utm_', column) is not None]
df_dict['pageviews'].head()

,is_article,action,article_id,category,host,name,remp_session_id,signed_in,social,tags,time,title,token,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
0,False,load,,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,,NaN,,,2018-01-13 00:00:00.076000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/autor/bella/,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
1,True,load,294893.0,slovensko,npress-influxdb-n2.ewhs.bntb.net,pageviews,,NaN,,vybuchy-v-parizi,2018-01-13 00:00:00.078000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/294893/fico-bezpecnost-u-na...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
2,True,load,544744.0,slovensko,npress-influxdb-n2.ewhs.bntb.net,pageviews,,NaN,,,2018-01-13 00:00:00.078000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/544744/ciznar-a-matovic-si-...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
3,True,load,704078.0,komentare,npress-influxdb-n2.ewhs.bntb.net,pageviews,,NaN,,,2018-01-13 00:00:00.078000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/704078/za-boha-a-za-narod-b...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
4,True,load,953687.0,nezaradene,npress-influxdb-n2.ewhs.bntb.net,pageviews,,NaN,,rozhovory,2018-01-13 00:00:00.078000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/953687/ako-evanjelici-sami-...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,


In [375]:
# This function assumes all missing values are either NaN or ''
def get_missing_values_share(table_name: str) -> pd.DataFrame:
    missing_values_share = pd.Series(index=df_dict[table_name].columns)
    num_df_rows = len(df_dict[table_name])
    for column in df_dict[table_name].columns:
        non_na_rows = df_dict[table_name][column].dropna()
        if str(non_na_rows.dtype) == 'object':
            num_relevant_values = len(non_na_rows[non_na_rows != ''])
        else:
            num_relevant_values = len(non_na_rows)
        missing_values_share[column] = num_relevant_values / num_df_rows
    
    return missing_values_share

In [376]:
get_missing_values_share('pageviews')

is_article         1.000000
action             1.000000
article_id         0.525908
category           0.999994
host               1.000000
name               1.000000
remp_session_id    0.011186
signed_in          0.011230
social             0.104669
tags               0.159106
time               1.000000
title              0.000005
token              1.000000
url                1.000000
user_agent         0.000002
user_id            1.000000
utm_campaign       0.012994
utm_content        0.010596
utm_medium         0.013796
utm_source         0.013797
dtype: float64

Look at signed_in values

In [338]:
df_dict['pageviews'][df_dict['pageviews']['signed_in'] == 1].head(3)

,is_article,action,article_id,category,host,name,remp_session_id,signed_in,social,tags,time,title,token,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
796,False,load,nan,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,74014bf5-abc9-4d25-8aec-baea54ab0b61,1.0,,,2018-01-17 00:11:32.094000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/,,122004,,,,
983,False,load,nan,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,74014bf5-abc9-4d25-8aec-baea54ab0b61,1.0,,,2018-01-17 00:14:51.644000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/,,122004,,,,
4230,False,load,nan,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,d8e82e4b-9a7e-4505-9614-3cc9a4350244,1.0,,,2018-01-17 01:42:16.587000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/,,39848,,,,


In [339]:
df_dict['pageviews'][df_dict['pageviews']['signed_in'] == 0].head(3)

,is_article,action,article_id,category,host,name,remp_session_id,signed_in,social,tags,time,title,token,url,user_agent,user_id,utm_campaign,utm_content,utm_medium,utm_source
0,False,load,nan,pageview,npress-influxdb-n2.ewhs.bntb.net,pageviews,,0.0,,,2018-01-17 00:00:00.076000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/veda/page/5/,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
3,True,load,253261.0,zdravie,npress-influxdb-n2.ewhs.bntb.net,pageviews,,0.0,,,2018-01-17 00:00:00.078000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/253261/opice-sa-vekom-zhors...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,
4,True,load,269636.0,ekonomika,npress-influxdb-n2.ewhs.bntb.net,pageviews,,0.0,,,2018-01-17 00:00:00.078000,,1a8feb16-3e30-4f9b-bf74-20037ea8505a,https://dennikn.sk/269636/platia-na-poste-seko...,,2dfb3791-a51c-4c7a-819d-f7a94a29c015,,,,


Tags by whether the url is an article

In [350]:
df_dict['pageviews'][df_dict['pageviews']['is_article'] == True]['tags'].value_counts().head()

                          3387809
rozhovory                  230977
ceske-volby-prezidenta     121159
hoaxy-a-propaganda          91298
slovenske-myty              75090
Name: tags, dtype: int64

In [397]:
unique_articles = df_dict['pageviews'].loc[
    df_dict['pageviews']['is_article'] == True, 
    ['article_id', 'tags']].drop_duplicates()
print('share articles without tags', 
      len(unique_articles[unique_articles['tags'] == '']) / len(unique_articles))

share articles without tags 0.7579430670339761


In [349]:
df_dict['pageviews'][df_dict['pageviews']['is_article'] != True]['tags'].value_counts()

    4378740
Name: tags, dtype: int64

Look at article_id by is_article

In [380]:
df_dict['pageviews'].loc[df_dict['pageviews']['article_id'] != '', 'is_article'].value_counts()

True    4857323
Name: is_article, dtype: int64

In [381]:
df_dict['pageviews'].loc[df_dict['pageviews']['article_id'] == '', 'is_article'].value_counts()

False    4378740
Name: is_article, dtype: int64